In [2]:
mutable struct Account
    account_number::String
    balance::Float64
    date_opened::Date
end

account_number(a::Account)=a.account_number
balance(a::Account)=a.balance
date_opened(a::Account)=a.date_opened

function deposit!(a::Account,amount::Real)
    a.balance+=amount
    return a.balance
end

function withdraw!(a::Account,amount::Real)
    a.balance-=amount
    return a.balance
end

function transfer!(from::Account,to::Account,amount::Real)
    withdraw!(from,amount)
    deposit(to,amount)
    return amount
end


LoadError: UndefVarError: Date not defined

In [ ]:
struct SavingAccount
    acct::Account
    interest_rate::Float64
    #constructor
    SavingAccount(account_number,balance,date_opened)=new(Account(account_number,balance,date_opened),interest_rate)
end

using Lazy: @forward